<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verificaiton/batcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import pickle
import random

import import_ipynb
from constants import NUM_FBANK, HOP_LENGTH, NUM_PER_SPEAKER, NUM_FRAME

1


In [ ]:
class naive_batcher:
    def __init__(self, input_dir, output_dir):
        self.name = 'naive_batcher'
        self.input_dir = input_dir
        self.output_dir = output_dir
        
        if len(os.listdir(self.output_dir))==0: self.generate()
        else: print('load exist data')
        self.filename = os.listdir(self.output_dir)[0]
        self.num_speaker = int(self.filename.split('_')[-4])
    
    def generate(self, num_frame=NUM_FRAME, num_per_speaker=NUM_PER_SPEAKER):
        #if len(os.listdir(self.output_dir))!=0: error()
        X = []
        Y = []
        speaker_id = 0        
        
        for speaker in os.listdir(self.input_dir):
            X_speaker = []
            for utt in os.listdir(self.input_dir+'/'+speaker):
                spec = np.load(self.input_dir+'/'+speaker+'/'+utt)
                for i in range(0,spec.shape[0]-num_frame+1,num_frame): X_speaker.append(spec[i:i+num_frame,:])
                if len(X_speaker)>=num_per_speaker: break
            if len(X_speaker)<num_per_speaker: continue
            X.extend(X_speaker[:num_per_speaker])
            Y.extend([speaker_id]*num_per_speaker)
            speaker_id+=1
        X = np.array(X)
        Y = np.array(Y)
        
        # shuffle utterances
        shuffle_idx = np.arange(X.shape[0])
        np.random.shuffle(shuffle_idx)
        X = X[shuffle_idx]
        Y = Y[shuffle_idx]
        
        # split train/test
        split_idx = int(X.shape[0]*0.8)
        X_train, X_test, Y_train, Y_test = X[:split_idx], X[split_idx:], Y[:split_idx], Y[split_idx:]
        
        # print information
        print()
        print('generate train/test dataset')
        print('shape:', X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
        
        # save
        XY = (X_train, X_test, Y_train, Y_test)
        with open(self.output_dir+'/'+f"{speaker_id}_{NUM_PER_SPEAKER}_{NUM_FRAME}_{NUM_FBANK}.npy",'wb') as f:
            pickle.dump(XY,f)
        
    def load_data(self):
        with open(self.output_dir+'/'+self.filename,"rb") as f:
            return pickle.load(f)

NameError: name 'NUM_FRAME' is not defined

In [ ]:
def get_batcher(batcher_name, input_dir, output_dir):
    if batcher_name=='naive_batcher': return naive_batcher(input_dir, output_dir)
    
    